In [57]:
import json
import bz2
import time
import datetime
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import scale
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [4]:
df = features
y = df['radiant_win']
X = df.drop(columns=['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'])
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Columns: 102 entries, start_time to dire_first_ward_time
dtypes: float64(12), int64(90)
memory usage: 76.4 MB


In [5]:
X_notna = X.isna().sum()
X_notna[X_notna != 0]

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

first_blood_time, first_blood_team и first_blood_player1 - говорит что первая кровь не всегде делается за первые 5 минут. => предположу что в таких играх будет влиять кол-во общего золота и герои.
first_blood_player2 - предположу что "второй игрок, причастный к событию" это тот кто помог всетаки. Так что нормально что числа не совпадают, так как событие может быть 1 на 1(1/3 от всех матчей).
radiant_bottle_time и dire_bottle_time - означает что либо вообще непокупают бутылку или все очень плохо у мидера. Но вообще сильно от героя зависит есть те кто может сразу и кому вообще не нужна. 
radiant_courier_time и dire_courier_time - не покупают курьера в начале => все очень плохо у саппорта и всей команды. Или саппорта нет или он дебил.
radiant_flying_courier_time и dire_flying_courier_time - не могут улучить курьера => все очень плохо у саппорта и всей команды.
radiant_first_ward_time и dire_first_ward_time - не покупают варды в начале => все очень плохо у саппорта и всей команды. НО варды еще покупают и другие обычно.

In [6]:
values = {'first_blood_time': 3000, 
          'first_blood_team': -1, 
          'first_blood_player1': -1,  
          'first_blood_player2': -1,
          'radiant_bottle_time': 3000,
          'radiant_courier_time': 3000, 
          'radiant_flying_courier_time': 3000, 
          'radiant_first_ward_time': 3000, 
          'dire_bottle_time': 3000,
          'dire_courier_time': 3000,
          'dire_flying_courier_time': 3000,
          'dire_first_ward_time': 3000,} 
#Нигде тут нельзя заменять на среднее так как нам важно что событие не произошло в превые 5 мин.
#НО вот если бы мы знали среднее после 5 мин и могли заменить на него стало бы лучше.
X = X.fillna(value=values)

In [7]:
X_notna = X.isna().sum()
X_notna[X_notna != 0]

Series([], dtype: int64)

In [8]:
k_fold = KFold(n_splits=5, random_state=42, shuffle=True) 

for i in range(10, 100, 10):
    start_time = datetime.datetime.now()

    model = GradientBoostingClassifier(n_estimators=i, random_state=42)

    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  


    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)
roc_auc
[0.66383799 0.66635457 0.66360048 0.66517303 0.66516222]
Среднее: 0.66483 
Стандартное отклонение: 0.00200959 
Time elapsed: 0:00:16.982618
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort

In [9]:
for i in range(100, 200, 10):
    start_time = datetime.datetime.now()

    model = GradientBoostingClassifier(n_estimators=i, random_state=42)

    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  


    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)
[0.70523705 0.70633163 0.70458526 0.70829408 0.70802096]
Среднее: 0.70649 
Стандартное отклонение: 0.00294231 
Time elapsed: 0:02:25.838389
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=110,
              presort='auto

In [10]:
start_time = datetime.datetime.now()

model = GradientBoostingClassifier(n_estimators=1000, random_state=42)

print(model)
    
scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
print(scores)
print("Среднее: %0.5f " % (scores.mean()))
print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  

+
print('Time elapsed:', end=' ')
print(datetime.datetime.now() - start_time)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)
[0.72431535 0.72288417 0.72351451 0.72770389 0.72451518]
Среднее: 0.72459 
Стандартное отклонение: 0.00332792 
Time elapsed: 0:25:21.641317


Лучший рез
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)
[0.72431535 0.72288417 0.72351451 0.72770389 0.72451518]
Среднее: 0.72459 
Стандартное отклонение: 0.00332792 
Time elapsed: 0:25:21.641317

Логистическая регрессия

#1

In [70]:
df = features

In [71]:
drop_columns = ['duration',
                'radiant_win',
                'tower_status_radiant',
                'tower_status_dire',
                'barracks_status_radiant',
                'barracks_status_dire',
               ]

y = df['radiant_win']
X = df.drop(columns=drop_columns)

values = {'first_blood_time': 3000, 
          'first_blood_team': -1, 
          'first_blood_player1': -1,  
          'first_blood_player2': -1,
          'radiant_bottle_time': 3000,
          'radiant_courier_time': 3000, 
          'radiant_flying_courier_time': 3000, 
          'radiant_first_ward_time': 3000, 
          'dire_bottle_time': 3000,
          'dire_courier_time': 3000,
          'dire_flying_courier_time': 3000,
          'dire_first_ward_time': 3000,} 
X = X.fillna(value=values)

In [72]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [73]:
for i in np.arange(0.1, 1.1, 0.1):
    start_time = datetime.datetime.now()

    model = LogisticRegression(random_state=42, C=i, penalty='l2')
    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  


    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71387045 0.71599013 0.7156223  0.71667172 0.71764179]
Среднее: 0.71596 
Стандартное отклонение: 0.00250058 
Time elapsed: 0:00:10.781543
LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71386827 0.71598805 0.71561685 0.71667252 0.71763757]
Среднее: 0.71596 
Стандартное отклонение: 0.00250052 
Time elapsed: 0:00:10.773095
LogisticRegression(C=0.30000000000000004, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', ran

LogisticRegression(C=0.91, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71386725 0.71598743 0.71561321 0.71667409 0.71763492]
Среднее: 0.71596 
Стандартное отклонение: 0.00250052 
Time elapsed: 0:00:11.117423

In [74]:
for i in range(1, 11, 1):
    start_time = datetime.datetime.now()

    model = LogisticRegression(random_state=42, C=i, penalty='l2')
    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  


    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71386716 0.71598715 0.71561309 0.7166742  0.71763469]
Среднее: 0.71596 
Стандартное отклонение: 0.00250050 
Time elapsed: 0:00:10.943264
LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71386702 0.71598696 0.71561254 0.71667485 0.71763386]
Среднее: 0.71596 
Стандартное отклонение: 0.00250035 
Time elapsed: 0:00:10.917588
LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='

#2

In [62]:
df = features

In [63]:
drop_columns = ['duration',
                'radiant_win',
                'tower_status_radiant',
                'tower_status_dire',
                'barracks_status_radiant',
                'barracks_status_dire',
                'lobby_type',
                'r1_hero',
                'r2_hero',
                'r3_hero',
                'r4_hero',
                'r5_hero',
                'd1_hero',
                'd2_hero',
                'd3_hero',
                'd4_hero',
                'd5_hero',
               ]

y = df['radiant_win']
X = df.drop(columns=drop_columns)

In [64]:
values = {'first_blood_time': 3000, 
          'first_blood_team': -1, 
          'first_blood_player1': -1,  
          'first_blood_player2': -1,
          'radiant_bottle_time': 3000,
          'radiant_courier_time': 3000, 
          'radiant_flying_courier_time': 3000, 
          'radiant_first_ward_time': 3000, 
          'dire_bottle_time': 3000,
          'dire_courier_time': 3000,
          'dire_flying_courier_time': 3000,
          'dire_first_ward_time': 3000,} 
X = X.fillna(value=values)

In [65]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [69]:
for i in np.arange(0.1, 2.0, 0.1):
    start_time = datetime.datetime.now()

    model = LogisticRegression(random_state=42, C=i, penalty='l2')
    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  

    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71374904 0.71603673 0.71567785 0.7166047  0.71784607]
Среднее: 0.71598 
Стандартное отклонение: 0.00267494 
Time elapsed: 0:00:09.903605
LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.7137453  0.71603349 0.71567319 0.71660705 0.71784147]
Среднее: 0.71598 
Стандартное отклонение: 0.00267569 
Time elapsed: 0:00:09.600655
LogisticRegression(C=0.30000000000000004, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', ran

LogisticRegression(C=1.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.71374345 0.71603291 0.71567084 0.71660965 0.71783923]
Среднее: 0.71598 
Стандартное отклонение: 0.00267638 
Time elapsed: 0:00:09.405575

#3

In [108]:
df = features

In [109]:
d = df['d1_hero'].append([df['d2_hero'], df['d3_hero'], df['d4_hero'], df['d5_hero'], df['r1_hero'], df['r2_hero'], df['r3_hero'], df['r4_hero'], df['r5_hero']] , ignore_index=True)
#Вообще соединять не нужно было ,но вдруг... 
N = d.unique().size 
N

108

In [110]:
N = d.unique().max()
N

112

In [111]:
np.sort(d.unique())
#Тут пропущенный id

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

In [112]:
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1


In [113]:
df_pick = pd.DataFrame(data=X_pick, index=df.index)
df_pick.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
match_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
drop_columns = ['duration',
                'radiant_win',
                'tower_status_radiant',
                'tower_status_dire',
                'barracks_status_radiant',
                'barracks_status_dire',
                'lobby_type',
                'r1_hero',
                'r2_hero',
                'r3_hero',
                'r4_hero',
                'r5_hero',
                'd1_hero',
                'd2_hero',
                'd3_hero',
                'd4_hero',
                'd5_hero',
               ]

y = df['radiant_win']
X = df.drop(columns=drop_columns)

values = {'first_blood_time': 3000, 
          'first_blood_team': -1, 
          'first_blood_player1': -1,  
          'first_blood_player2': -1,
          'radiant_bottle_time': 3000,
          'radiant_courier_time': 3000, 
          'radiant_flying_courier_time': 3000, 
          'radiant_first_ward_time': 3000, 
          'dire_bottle_time': 3000,
          'dire_courier_time': 3000,
          'dire_flying_courier_time': 3000,
          'dire_first_ward_time': 3000,} 
X = X.fillna(value=values)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Data columns (total 91 columns):
start_time                     97230 non-null int64
r1_level                       97230 non-null int64
r1_xp                          97230 non-null int64
r1_gold                        97230 non-null int64
r1_lh                          97230 non-null int64
r1_kills                       97230 non-null int64
r1_deaths                      97230 non-null int64
r1_items                       97230 non-null int64
r2_level                       97230 non-null int64
r2_xp                          97230 non-null int64
r2_gold                        97230 non-null int64
r2_lh                          97230 non-null int64
r2_kills                       97230 non-null int64
r2_deaths                      97230 non-null int64
r2_items                       97230 non-null int64
r3_level                       97230 non-null int64
r3_xp                          97230 non-null int64
r3_gol

In [115]:
X = pd.merge(X, df_pick, left_index=True, right_index=True, how='outer')
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 114406
Columns: 203 entries, start_time to 111
dtypes: float64(124), int64(79)
memory usage: 153.8 MB


In [116]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [117]:
for i in np.arange(0.1, 2.0, 0.1):
    start_time = datetime.datetime.now()

    model = LogisticRegression(random_state=42, C=i, penalty='l2')
    print(model)
    
    scores = cross_val_score(model, X, y, cv=k_fold, scoring="roc_auc")
    print(scores)
    print("Среднее: %0.5f " % (scores.mean()))
    print("Стандартное отклонение: %0.8f " % (scores.std() * 2))  

    print('Time elapsed:', end=' ')
    print(datetime.datetime.now() - start_time)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.74887603 0.75319956 0.74938933 0.75582093 0.75135239]
Среднее: 0.75173 
Стандартное отклонение: 0.00511169 
Time elapsed: 0:00:19.196522
LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.74887592 0.75319961 0.7493889  0.75581668 0.75134786]
Среднее: 0.75173 
Стандартное отклонение: 0.00510946 
Time elapsed: 0:00:19.508623
LogisticRegression(C=0.30000000000000004, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', ran

LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.74887592 0.75319961 0.7493889  0.75581668 0.75134786]
Среднее: 0.75173 
Стандартное отклонение: 0.00510946 
Time elapsed: 0:00:19.508623